# Exploration of Published Models

The tumour growth inhibiting effects of Erlotinib and Gefitinib were modelled with a population PKPD model in [1]. A population PKPD model is a hierarchical model which consists of a structural model, a population model, and an error model. Each sub-model captures a different aspect of the tumour growth inhibition biology, and is parametrised by a set of parameters. In this notebook, we review the population PKPD model structure presented in [1]. We further explore the different treatment strategies with Erlotinib and Gefitinib *in silico* for two types of cancers based on the reported model parameters. 

Note that in [1] it was argued that the PKPD properties of Erlotinib and Gefinitib are structurally the same. That is why in the following we introduce just one PKPD model structure and review the drug-specific model parameters in a subsequent step.

## Structural model

The structural model reported in [1] is a system of ordindary differential and algebraic equations. It combines a mechanistic description of the pharmacokinetics (PK) and the pharmacodynamics (PD) of the compound, as well as an empirical model of the tumour growth.

The distribution of the compound in the mouse system after an oral dose is modelled by a one-compartment PK model with indirect drug administration

\begin{align*}
\frac{\text{d}A_d}{\text{d}t} = r_d(t) - k_{a}A_d,\qquad \frac{\text{d}A_c}{\text{d}t} = k_{a} A_d - k_e A_c,\qquad C_c = \frac{A_c}{V_c}.
\end{align*}

Here, $A_d$ is the amount of the compound in the dose compartment, which may be interpreted as the gut compartment, while $A_c$ is the amount in the central compartment which represents the drug in the blood plasma. 

The drug is orally administered. As a result, its uptake in the dose compartment may be modelled by a time dependent dose rate $r_d$. The drug then diffuses at a constant rate $k_a$ (absorption rate) into the central compartment, where its clearance through the liver out of the system is modelled with a linear elimination rate $k_e$. The concentration of the compound in the central compartment $C_c$ can be derived from its amount and its effective volume of distribution in the central compartment $V_c$.

The tumour growth is modelled by an empirical model. In absence of the drug the tumour is assumed to grow exponentially for small tumour volumes, and to grow linearly for large tumour volumes. The growth inhibiting effects of the compound are assumed to be proportional to its concentration in the central compartment and the volume of the tumour

\begin{align*}
\frac{\text{d}V^s_T}{\text{d}t} = \frac{2\lambda _0\lambda _1 V^s_T}{2\lambda _0 V^s_T + \lambda _1} - \kappa C_cV^s_T.
\end{align*}

Here, $V^s_T$ is the predicted tumour volume by the structural model, $\lambda _0$ is the exponential growth rate, $\lambda _1$ is the linear growth rate, and $\kappa $ is the growth inhibiting potency of the compound.

In [1], the dose and central compartment were assumed to initially contain no amount of the compound, $A_d(t=0)=0$ and $A_c(t=0)=0$. The structural model is then parameterised by seven parameters that determine the PKPD of the drug

\begin{align*}
\psi = (V_0, k_a, k_e, V_c, \lambda _0, \lambda _1, \kappa ),
\end{align*}

where $V_0$ is the initial tumour volume, $V^s_T(t=0)=V_0$. Note that in this model $r_d$ is determined by the dosing regimen.

We have implemented the structural model in a separate python [module](https://github.com/DavAug/ErlotinibGefitinib/blob/master/pkpd/model.py), which generates a [myokit](http://myokit.org/) representation of the model with `pkpd.model.create_pktgi_model()`. [myokit](http://myokit.org/) allows us to solve the system of ODEs numerically.

In [1]:
#
# Create structural model.
#

import pkpd.model as m


# Create model
model = m.create_pktgi_model()

# Set route of administration to 'oral dosing'
model.set_roa(dose_comp='central', indirect=True)

# Show model
print(model.code())

[[model]]
# Initial values
central.amount   = 0
central.volume_t = 0
depot.amount     = 0

[central]
dot(amount) = -k_e * amount + depot.k_a * depot.amount
    in [mg]
conc = amount / volume_c
    in [mg/L]
k_e = 0
    in [1/day]
kappa = 0
    in [L/mg/day]
lambda_0 = 0
    in [1/day]
lambda_1 = 1
    in [cm^3/day]
time = 0 bind time
    in [day]
volume_c = 1
    in [L]
dot(volume_t) = 2 * (lambda_0 * (lambda_1 * volume_t)) / (2 * (lambda_0 * volume_t) + lambda_1) - kappa * (conc * volume_t)
    in [cm^3]

[depot]
dot(amount) = -k_a * amount + dose_rate * regimen
    in [mg]
dose_rate = 0
    in [mg/day]
k_a = 0
    in [1/day]
regimen = 0 bind pace
    in [1]




## Error model

The expected random deviations of the observed tumour volumes from the structural model predictions $V^s_T$ are modelled in [1] by a combined error model. A combined error model assumes that the deviations $\varepsilon $ of the observable tumour volume $V_T$ from the structural model predictions $V^s_T$ may be modelled by a base-level Gaussian noise, and a Gaussian noise contribution whose standard deviation grows relative to the structural model predictions

\begin{equation*}
    \varepsilon = \left(\sigma _{\text{base}} + \sigma _{\text{rel}} V^s_T\right) \varepsilon _n.
\end{equation*}

Here, $\sigma _{\text{base}}$ is the standard deviation of the base-level noise, and $\sigma _{\text{rel}}$ is the standard deviation relative to $V^s_T$. $\varepsilon _n$ is a standard Gaussian random variable $\mathcal{N}(0, 1)$. As a result, the model predictions of the tumour volume are Gaussian-distributed, centered at the structural model predictions $V^s_T = V^s_T(t, r_d; \psi)$ with a standard deviation $\sigma _{\text{tot}} = \sigma _{\text{base}} + \sigma _{\text{rel}} V^s_T$

\begin{equation*}
    V_T \sim \mathcal{N}(V^s_T, \sigma ^2_{\text{tot}}).
\end{equation*}

The error model introduces two further parameters to the model

\begin{equation*}
    \theta _V = (\sigma _{\text{base}}, \sigma _{\text{rel}}),
\end{equation*}

such that at this point the PKPD model has nine parameters ($\psi$, $\theta _V$). This model can be used to describe and predict tumour growth curves of individual mice, whose physiology is parameterised by a fixed set of model parameters ($\psi$, $\theta _V$).

## Population model

Generally, the structural model parameters $\psi$ can be split into system-specific and process-specific parameters. System-specific parameters are expected to vary between individuals, while process-specfic parameters are assumed to be constant across individuals. In [1] all structural model parameters $\psi $ are assumed to vary between individuals, except the absorption rate $k_a$. The population model defines the distribution of system-specific parameters in the population. 

In [1] all $\psi $ except the absorption rate $k_a$ are assumed to be [log-normally distributed](https://en.wikipedia.org/wiki/Log-normal_distribution) in the mouse population

\begin{equation*}
    \log \psi _i \sim \mathcal{N} (\log \mu _i, \sigma ^2_i), \quad \psi _i \neq k_a,
\end{equation*}

where $\mu _i $ is the median of $\psi _i$ in the population, and $\sigma _i$ the standard deviation of $\log \psi _i$ in the population. Note that $\mu _i$ is of linear scale and $\sigma _i$ is of logarithmic scale. Their values can therefore not be directly compared. 

The introduction of a population model replaces all system-specific structural model parameters by a distribution which is parameterised by the population median $\mu _i $ and the log-scale standard deviation $\sigma _i$. Defining the population parameters as 

\begin{equation*}
    \theta _{\psi } = (\mu _{V_0}, \sigma _{V_0}, k_a, \mu _{k_e}, \sigma _{k_e}, \mu _{V_c}, \sigma _{V_c}, \mu _{\lambda _0}, \sigma _{\lambda _0}, \mu _{\lambda _1}, \sigma _{\lambda _1}, \mu _{\kappa}, \sigma _{\kappa}),
\end{equation*}

the full population tumour growth inhibition PKPD model of Erlotinib and Gefitinib is parameterised by 15 parameters each, $(\theta _{\psi}, \theta _V)$.

## Parameter estimates

In [1] the model parameters $(\theta _{\psi}, \theta _V)$ for the PKPD of Erlotinib and Gefitinib have been inferred for patient-derived LXF A677 explants impanted in mice, and cell line-derived VXF A431 xenografts implanted in mice, see Table 1, p. 3117 in [1]. In general each drug-tumour type combination has an independent set of model parameters. However, here the absorption rate was fixed across all four models, and the tumour volume related parameters $(\mu _{V_0}, \sigma _{V_0}, \mu _{\lambda _0}, \sigma _{\lambda _0}, \mu _{\lambda _1}, \sigma _{\lambda _1}, \sigma _{\text{base}}, \sigma _{\text{rel}})$ were assumed to depend only on the tumour type.

Note that the reported "Estimate" for a parameter $\psi _i$ corresponds to the median of the population distribution $\mu _i$, while the interindividual variability "Var" is equivalent to the log-scale population standard deviation $\sigma _i$. The provided relative standard error (RSE) is the standard deviation of the estimates relative to their value (assuming a Gaussian error of the estimates for $\mu _i$ and $\sigma _i$). For later convenience we will translate the RSE values back into absolute standard deviations. For more details on Monolix's conventions please have a look at its [documentation](http://monolix.lixoft.com/data-and-models/individualdistribution/).

In [2]:
#
# Estimates from Eigenmann et. al. [1] for the Erlotinib and Gefitinib PKPD in LXF A677 and VXF A431.
#

import pandas as pd


# Set display of floats in pandas dataframes to 3 decimals
pd.options.display.float_format = '{:,.3f}'.format

# Create pandas dataframe for LXF A677 Erlotinib model parameters
lxf_erlotinib_estimates = pd.DataFrame(
    data={
        'mu V_0 in cm^3': [0.122, 0.122 * 0.05],
        'sigma V_0 in dimless': [0.368, 0.122 * 0.05],
        'k_a in 1/day': [55.0, None],
        'mu V_c in L': [0.127, 0.127 * 0.15],
        'sigma V_c in dimless': [0.251, None],
        'mu k_e in 1/day': [7.56, 7.56 * 0.1],
        'sigma k_e in dimless': [0.332, 0.332 * 0.39],
        'mu lambda_0 in 1/day': [0.0971, 0.0971 * 0.08],
        'sigma lambda_0 in dimless': [0.456, 0.456 * 0.13],
        'mu lambda_1 in cm^3/day': [0.127, 0.127 * 0.13],
        'sigma lambda_1 in dimless': [0.710, None],
        'mu kappa in L/mg/day': [0.117, 0.117 * 0.17],
        'sigma kappa in dimless': [0.654, 0.654 * 0.21],
        'sigma base in cm^3': [0.0141, 0.0141 * 0.09],
        'sigma rel in dimless': [0.0907, 0.0907 * 0.06]},
    index=['estimate', 'standard deviation'])

# Show dataframe
print('Table 1: Erlotinib LXF A677')
display(lxf_erlotinib_estimates)

# Create pandas dataframe for VXF A341 Erlotinib model parameters
vxf_erlotinib_estimates = pd.DataFrame(
    data={
        'mu V_0 in cm^3': [0.114, 0.114 * 0.06],
        'sigma V_0 in dimless': [0.423, 0.423 * 0.11],
        'k_a in 1/day': [55.0, None],
        'mu V_c in L': [0.120, 0.120 * 0.06],
        'sigma V_c in dimless': [0.190, 0.190 * 0.23],
        'mu k_e in 1/day': [7.85, 7.85 * 0.07],
        'sigma k_e in dimless': [0.235, 0.235 * 0.26],
        'mu lambda_0 in 1/day': [0.0290, 0.0290 * 0.06],
        'sigma lambda_0 in dimless': [0.321, 0.321 * 0.14],
        'mu lambda_1 in cm^3/day': [0.298, 0.298 * 0.37],
        'sigma lambda_1 in dimless': [0.68, None],
        'mu kappa in L/mg/day': [0.0401, 0.0401 * 0.16],
        'sigma kappa in dimless': [0.585, 0.585 * 0.21],
        'sigma base in cm^3': [0.00432, 0.00432 * 0.3],
        'sigma rel in dimless': [0.139, 0.139 * 0.05]},
    index=['estimate', 'standard deviation'])

# Show dataframe
print('Table 2: Erlotinib VXF A341')
display(vxf_erlotinib_estimates)

# Create pandas dataframe for LXF A677 Gefitinib model parameters
lxf_gefitinib_estimates = pd.DataFrame(
    data={
        'mu V_0 in cm^3': [0.122, 0.122 * 0.05],
        'sigma V_0 in dimless': [0.368, 0.122 * 0.05],
        'k_a in 1/day': [55.0, None],
        'mu V_c in L': [1.40, 1.40 * 0.11],
        'sigma V_c in dimless': [0.0278, None],
        'mu k_e in 1/day': [3.87, 3.87 * 0.10],
        'sigma k_e in dimless': [0.352, 0.352 * 0.20],
        'mu lambda_0 in 1/day': [0.0971, 0.0971 * 0.08],
        'sigma lambda_0 in dimless': [0.456, 0.456 * 0.13],
        'mu lambda_1 in cm^3/day': [0.127, 0.127 * 0.13],
        'sigma lambda_1 in dimless': [0.710, None],
        'mu kappa in L/mg/day': [0.430, 0.430 * 0.12],
        'sigma kappa in dimless': [0.361, 0.361 * 0.26],
        'sigma base in cm^3': [0.0141, 0.0141 * 0.09],
        'sigma rel in dimless': [0.0907, 0.0907 * 0.06]},
    index=['estimate', 'standard deviation'])

# Show dataframe
print('Table 3: Gefitinib LXF A677')
display(lxf_gefitinib_estimates)

# Create pandas dataframe for VXF A341 Gefitinib model parameters
vxf_gefitinib_estimates = pd.DataFrame(
    data={
        'mu V_0 in cm^3': [0.114, 0.114 * 0.06],
        'sigma V_0 in dimless': [0.423, 0.423 * 0.11],
        'k_a in 1/day': [55.0, None],
        'mu V_c in L': [0.472, 0.472 * 0.14],
        'sigma V_c in dimless': [0.466, 0.466 * 0.20],
        'mu k_e in 1/day': [6.10, 6.10 * 0.10],
        'sigma k_e in dimless': [0.228, 0.228 * 0.22],
        'mu lambda_0 in 1/day': [0.0290, 0.0290 * 0.06],
        'sigma lambda_0 in dimless': [0.321, 0.321 * 0.14],
        'mu lambda_1 in cm^3/day': [0.298, 0.298 * 0.37],
        'sigma lambda_1 in dimless': [0.68, None],
        'mu kappa in L/mg/day': [0.114, 0.114 * 0.15],
        'sigma kappa in dimless': [0.568, 0.568 * 0.20],
        'sigma base in cm^3': [0.00432, 0.00432 * 0.3],
        'sigma rel in dimless': [0.139, 0.139 * 0.05]},
    index=['estimate', 'standard deviation'])

# Show dataframe
print('Table 4: Gefitinib VXF A341')
display(vxf_gefitinib_estimates)

Table 1: Erlotinib LXF A677


,mu V_0 in cm^3,sigma V_0 in dimless,k_a in 1/day,mu V_c in L,sigma V_c in dimless,mu k_e in 1/day,sigma k_e in dimless,mu lambda_0 in 1/day,sigma lambda_0 in dimless,mu lambda_1 in cm^3/day,sigma lambda_1 in dimless,mu kappa in L/mg/day,sigma kappa in dimless,sigma base in cm^3,sigma rel in dimless
estimate,0.122,0.368,55.000,0.127,0.251,7.560,0.332,0.097,0.456,0.127,0.710,0.117,0.654,0.014,0.091
standard deviation,0.006,0.006,nan,0.019,nan,0.756,0.129,0.008,0.059,0.017,nan,0.020,0.137,0.001,0.005


Table 2: Erlotinib VXF A341


,mu V_0 in cm^3,sigma V_0 in dimless,k_a in 1/day,mu V_c in L,sigma V_c in dimless,mu k_e in 1/day,sigma k_e in dimless,mu lambda_0 in 1/day,sigma lambda_0 in dimless,mu lambda_1 in cm^3/day,sigma lambda_1 in dimless,mu kappa in L/mg/day,sigma kappa in dimless,sigma base in cm^3,sigma rel in dimless
estimate,0.114,0.423,55.000,0.120,0.190,7.850,0.235,0.029,0.321,0.298,0.680,0.040,0.585,0.004,0.139
standard deviation,0.007,0.047,nan,0.007,0.044,0.549,0.061,0.002,0.045,0.110,nan,0.006,0.123,0.001,0.007


Table 3: Gefitinib LXF A677


,mu V_0 in cm^3,sigma V_0 in dimless,k_a in 1/day,mu V_c in L,sigma V_c in dimless,mu k_e in 1/day,sigma k_e in dimless,mu lambda_0 in 1/day,sigma lambda_0 in dimless,mu lambda_1 in cm^3/day,sigma lambda_1 in dimless,mu kappa in L/mg/day,sigma kappa in dimless,sigma base in cm^3,sigma rel in dimless
estimate,0.122,0.368,55.000,1.400,0.028,3.870,0.352,0.097,0.456,0.127,0.710,0.430,0.361,0.014,0.091
standard deviation,0.006,0.006,nan,0.154,nan,0.387,0.070,0.008,0.059,0.017,nan,0.052,0.094,0.001,0.005


Table 4: Gefitinib VXF A341


,mu V_0 in cm^3,sigma V_0 in dimless,k_a in 1/day,mu V_c in L,sigma V_c in dimless,mu k_e in 1/day,sigma k_e in dimless,mu lambda_0 in 1/day,sigma lambda_0 in dimless,mu lambda_1 in cm^3/day,sigma lambda_1 in dimless,mu kappa in L/mg/day,sigma kappa in dimless,sigma base in cm^3,sigma rel in dimless
estimate,0.114,0.423,55.000,0.472,0.466,6.100,0.228,0.029,0.321,0.298,0.680,0.114,0.568,0.004,0.139
standard deviation,0.007,0.047,nan,0.066,0.093,0.610,0.050,0.002,0.045,0.110,nan,0.017,0.114,0.001,0.007


**Table 1-4:** Estimates and their uncertainties for the model parameters $(\theta _{\psi}, \theta _V)$ inferred in [1]. Table 1 and 2 show the inferred PKPD parameters for Erlotinib treating the transplanted LXF A677 (ardenocarcinoma of the lung) and VXF A431 (vulva cancer) tumours, respectively. Analogously, table 3 and 4 show the inferred PKPD parameters for Gefitinib treating transplanted LXF A677 and VXF A431 tumours. 

## Explore predictions for tumour growth inhibition

With these model parameters, we can simulate dose-response tumour growth curves for different Erlotinib and Gefitinib treatment strategies of LXF A677 lung cancer and VXF A431 vulva cancer. We first simulate the dose-response in a population of mice for 3 dosing regimens similar to the ones presented in [1]. We then explore the dose-response of a "median" mouse for a larger variety of dosing strategies.

The distribution of dose-response curves in the population can be approximated by repeatedly sampling individuals from the population distribution. Each 'virtual' mouse in the population is then fully characterised by a sample of structural model parameters $\psi _i \sim \mathcal{N}(\log \mu _i , \sigma _i^2)$ for all $i$. Denoting the structural model parameter set of an individual $j$ by 

\begin{align*}
\psi _j = (k_{a, j}, k_{e, j}, V_{c, j}, V_{0, j}, \lambda _{0, j}, \lambda _{1, j}, \kappa _j),
\end{align*}

we can sample the tumour growth predictions from the structural-error model $V_{T,j} = \mathcal{N}(V^s_{T,j}, \sigma ^2_{\text{tot}, j})$ , where the mean and the variance are determined by the individual's set of structural model parameters $\psi _j$, $V^s_{T,j} = V^s_{T}(t, r_d;\psi _j)$ and $\sigma _{\text{tot}, j} = \sigma _{\text{base}} + \sigma _{\text{rel}}V^s_{T,j}$. Note that the estimates reported in [1] are not exact and partly display significant uncertainty. From a Bayesian perspective we would therefore like to construct posterior distributions of the model parameters which reflect the uncertainty in their estimates. With just the estimates and their standard deviation, this is however not possible.

Denoting the estimates with a "hat", we thus construct the population dose-response curves according to

\begin{equation*}
    V_{T,j} = \mathcal{N}(V^s_{T,j}, \sigma ^2_{\text{tot}, j})\quad \text{with}\quad \log \psi _{ij} \sim \mathcal{N}(\log \hat{\mu} _i, \hat{\sigma}^2_i),
\end{equation*}

and $k_{a,j}=\hat{k}_{a}$, $\sigma _{\text{base}}= \hat{\sigma }_{\text{base}}$, $\sigma _{\text{rel}}=\hat{\sigma }_{\text{rel}}$, acknowledging the fact, that the uncertainty in the model predictions will be underestimated. 

To simulate the population behaviour, we sample $N=1000$ individuals from the PKPD model, and visualise their dose-response variation by computing percentiles of the samples at predefined time points. Note that this representation captures the spread of different tumour volumes in the population well, however, loses any information about individual growth curves. That is to say, that the median growth curve in the population does not actually describe the tumour growth of a median mouse in the population, but rather the median tumour volume at each sampled time point.

In [7]:
#
# Sampling mice from the Erlotinib LXF A677, Erlotinib VXF A431, Gefitinib LXF A677 and Gefitinib VXF A431 population.
#
# This cell needs the above defined parameter estimates 
# [lxf_erlotinib_estimates, vxf_erlotinib_estimates, lxf_gefitinib_estimates, vxf_gefitinib_estimates]
#

import numpy as np
from scipy.stats import lognorm


# Define population size
N = 1000

# Number of parameters that define mouse (noise params are not mouse specific)
n_params = 7  # [V_0, k_a, V_c, k_e, lambda_0, lambda_1, kappa]

# Define population parameters
populations = [
    lxf_erlotinib_estimates, 
    vxf_erlotinib_estimates,
    lxf_gefitinib_estimates, 
    vxf_gefitinib_estimates]
n_populations = len(populations)

# Create container for mice (n_populations, N, n_parameters)
mice_parameters = np.empty(shape=(n_populations, N, n_params))

# Sample mice for each of the populations
for pop_id, population in enumerate(populations):
    # Seed random number generator
    # (Effectively creates the same `virtual` mice in all populations, 
    # i.e. lxf mice 13 will have the same drug-independent growth 
    # parameters wether treated with Erlotinib or Gefitinib)
    np.random.seed(42)

    # Sample initial tumour volumes
    median = population.loc['estimate', 'mu V_0 in cm^3']
    std = population.loc['estimate', 'sigma V_0 in dimless']
    mice_parameters[pop_id, :, 0] = lognorm.rvs(scale=median, s=std, size=N)

    # Sample absorption rate
    mice_parameters[pop_id, :, 1] = np.full(shape=N,fill_value=population.loc['estimate', 'k_a in 1/day'])

    # Sample volume of distribution
    median = population.loc['estimate', 'mu V_c in L']
    std = population.loc['estimate', 'sigma V_c in dimless']
    mice_parameters[pop_id, :, 2] = lognorm.rvs(scale=median, s=std, size=N)

    # Sample elimination rate
    median = population.loc['estimate', 'mu k_e in 1/day']
    std = population.loc['estimate', 'sigma k_e in dimless']
    mice_parameters[pop_id, :, 3] = lognorm.rvs(scale=median, s=std, size=N)

    # Sample exponential growth rate
    median = population.loc['estimate', 'mu lambda_0 in 1/day']
    std = population.loc['estimate', 'sigma lambda_0 in dimless']
    mice_parameters[pop_id, :, 4] = lognorm.rvs(scale=median, s=std, size=N)

    # Sample linear growth rate
    median = population.loc['estimate', 'mu lambda_1 in cm^3/day']
    std = population.loc['estimate', 'sigma lambda_1 in dimless']
    mice_parameters[pop_id, :, 5] = lognorm.rvs(scale=median, s=std, size=N)

    # Sample potency
    median = population.loc['estimate', 'mu kappa in L/mg/day']
    std = population.loc['estimate', 'sigma kappa in dimless']
    mice_parameters[pop_id, :, 6] = lognorm.rvs(scale=median, s=std, size=N)


In [4]:
#
# Simulate tumour growth curves for simulated mice under different dosing strategies.
#
# Dosing strategies: 
# Daily oral administration from day 3 to (including) 16 of
#   0. No treatment
#   1. 25 mg/kg/day (amount drug per mouse weight per day)
#   2. 50 mg/kg/day
#   3. 100 mg/kg/day
# Following [1], all mice in the population are assumed to weigh 20 mg.
#
# This cell needs the above simulated mice parameters and the reference population parameters: 
# [mice_parameeters, populations]
# 

import myokit
from scipy.stats import norm

import pkpd.model as m

# Get population size and number of populations
n_populations = mice_parameters.shape[0]
N = mice_parameters.shape[1]

# Define dosing regimens
# Amount [mg], dose duration [day], start [day], period [day], multiplier [dimless]
mouse_mass = 0.02  # 20 mg
dosing_regimens = [
    [0, 1E-03, 3, 1, 14],  # No treatment
    [25 * mouse_mass, 1E-03, 3, 1, 14],
    [50 * mouse_mass, 1E-03, 3, 1, 14],
    [100 * mouse_mass, 1E-03, 3, 1, 14]]
n_regimens = len(dosing_regimens)

# Define percentiles for population summary
percentiles = [10, 25, 50, 75, 90]
n_percentiles = len(percentiles)

# Define simulation time points in days
times = np.linspace(0, 30, num=100)
n_times = len(times)

# Create container for simulation results 
# (n_populations, n_regimens, n_percentiles, n_times)
population_results = np.empty(shape=(
    n_populations, n_regimens, n_percentiles, n_times))

# Create structural model and set route of administration
model = m.create_pktgi_model()
model.set_roa(dose_comp='central', indirect=True)

# Create temporary container for structural model predictions of mice in a population
# (population size, n_times)
mice_dose_response = np.empty(shape=(N, n_times)) 

# Simulate dose-response in population for dosing regimens
for regimen_id, dosing_regimen in enumerate(dosing_regimens):
    # Get regimen
    amount, duration, start, period, multiplier = dosing_regimen

    # Create myokit dosing regimen
    model.set_regimen(
        amount=amount, duration=duration, start=start, period=period, multiplier=multiplier)
    _, protocol = model.dosing_regimen()

    # Create simulator
    simulator = myokit.Simulation(model, protocol)

    # Loop through populations
    # Recall: mice_parameters shape = (n_populations, N, n_parameters)
    for pop_id, population in enumerate(mice_parameters):
        
        # Loop through indiviuals in population
        # `mouse` contains parameters: [V_0, k_a, V_c, k_e, lambda_0, lambda_1, kappa]
        for mouse_id, mouse in enumerate(population):
            # Reset simulator
            simulator.reset()

            # Set initial state
            # central.amount, central.volume_t, depot.amount
            simulator.set_state([0, mouse[0], 0])

            # Set parameters
            simulator.set_constant('depot.k_a', mouse[1])
            simulator.set_constant('central.volume_c', mouse[2])
            simulator.set_constant('central.k_e', mouse[3])
            simulator.set_constant('central.lambda_0', mouse[4])
            simulator.set_constant('central.lambda_1', mouse[5])
            simulator.set_constant('central.kappa', mouse[6])

            # Define logged variable
            loggedVariable = 'central.volume_t'

            # Solve structural model
            output = simulator.run(times[-1] + 1, log=[loggedVariable], log_times=times)
            mice_dose_response[mouse_id, :] = np.array(output[loggedVariable])

        # Add error model predictions to structural model predictions
        # Get sigma base and sigma rel from dataframe
        sigma_base = populations[pop_id].loc['estimate', 'sigma base in cm^3']
        sigma_rel = populations[pop_id].loc['estimate', 'sigma rel in dimless']

        # Draw standard Gaussian random variable for each predicted tumour volume
        gaussian_rv = np.random.normal(loc=0.0, scale=1.0, size=mice_dose_response.shape)

        # Scale random variable according to error model
        # Eps = (sigma_base + sigma_rel * V^s_T) * gaussian_rv
        gaussian_rv *= sigma_base #(sigma_base + sigma_rel * mice_dose_response)

        # Overlay error model predictions on structural model predictions
        mice_dose_response += gaussian_rv

        # Compute population percentiles
        for perc_id, percentile in enumerate(percentiles):
            population_results[pop_id, regimen_id, perc_id, :] = np.percentile(
                a=mice_dose_response, q=percentile, axis=0)


In [8]:
#
# Convert dosing regimen into time series for illustration.
#

# Create container for dosing regimen time series
dosing_time_series = []

# Compute dosing regimen time series
for regimen in dosing_regimens:
    amount, duration, start, period, multiplier = regimen

    # Initialise container (start with zero amount at time 0, and continue to end of simulation)
    # Construct series by adding a point at dosing start and after finishing the dose.
    dose_series = np.zeros(shape=(2, 2*multiplier+2))

    # Construct dosing times
    dose_series[0, 1:-2:2] = np.arange(
        start=start, 
        stop=start+period*multiplier,
        step=period)  # time points of dose start
    dose_series[0, 2:-1:2] = dose_series[0, 1:-2:2] + duration  # time points of dose end
    
    # Compute dose amount administered
    dose_series[1, 1:-2:2] = np.arange(multiplier) * amount  # Start of dose
    dose_series[1, 2:-1:2] = np.arange(1, multiplier+1) * amount

    # Repeat last full amount for the rest of the simulation period
    dose_series[0, -1] = times[-1]
    dose_series[1, -1] = dose_series[1, -2]

    # Append to container
    dosing_time_series.append(dose_series)

### Plot population PKPD of Erlotinib and Gefitinib in LXF A677 mice 

In [11]:
#
# Illustrate population PKPD simulation.
#
# Needs above defined dosing regimens, percentiles, times, simulation results,
# and the population size:
# [dosing_regimens, percentiles, times, population_results, N]
#

# import plotly.io
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Create figure
#fig = go.Figure()
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_heights=[0.3, 0.7], vertical_spacing=0.05)

# Plot simulations for different dosing strategies
for dose_id, dose_series in enumerate(dosing_time_series):
    # Plot administered drug
    fig.add_trace(
        go.Scatter(
            x=dose_series[0, :],
            y=dose_series[1, :],
            name="Dosed amount",
            hovertemplate=
                    "<b></b>Cumulative amount of compound<br>" +
                    "Time: %{x:.0f} day<br>" +
                    "Amount: %{y:.01f} mg<br>" +
                    "<extra></extra>",
            line=dict(color='rgb(0, 0, 0)'),
            mode='lines',
            opacity=0.3,
            visible = dose_id == 0),  # Only show first dosing regimen
        row=1,
        col=1)

    # Plot data LXF A677 Erlotinib
    # Plot 10th percentile to 90th precentile
    fig.add_trace(
        go.Scatter(
            x=np.hstack([times,times[::-1]]),
            y=np.hstack([
                population_results[0, dose_id, 0, :], 
                population_results[0, dose_id, 4, ::-1]]),
            line=dict(width=1, color='rgb(51, 153, 255)'),
            fill='toself',
            legendgroup="LXF A677 Erlotinib",
            name="10th to 90th percentile",
            hoverinfo='name',
            showlegend=False,
            visible = dose_id == 0),  # Only show first dosing regimen
        row=2,
        col=1)

    # Plot 25th percentile to 75th precentile
    fig.add_trace(
        go.Scatter(
            x=np.hstack([times,times[::-1]]),
            y=np.hstack([
                population_results[0, dose_id, 1, :], 
                population_results[0, dose_id, 3, ::-1]]),
            line=dict(width=1, color='rgb(0, 128, 255)'),
            fill='toself',
            legendgroup="LXF A677 Erlotinib",
            name="25th to 75th percentile",
            hoverinfo='name',
            showlegend=False,
            visible = dose_id == 0),  # Only show first dosing regimen
        row=2,
        col=1)

    # Plot Median
    fig.add_trace(
        go.Scatter(
            x=times,
            y=population_results[0, dose_id, 2, :],
            legendgroup="LXF A677 Erlotinib",
            name="LXF A677 Erlotinib",
            hovertemplate=
                    "<b></b>Median<br>" +
                    "Population size: %d<br>" % N +
                    "Time: %{x:.02f} day<br>" +
                    "Tumour volume: %{y:.02f} cm^3<br>" +
                    "<extra></extra>",
            line=dict(color='rgb(0, 102, 204)'),
            visible = dose_id == 0),  # Only show first dosing regimen
        row=2,
        col=1)

    # Plot data LXF A677 Gefitinib
    # Plot 10th percentile to 90th precentile
    fig.add_trace(
        go.Scatter(
            x=np.hstack([times,times[::-1]]),
            y=np.hstack([
                population_results[2, dose_id, 0, :], 
                population_results[2, dose_id, 4, ::-1]]),
            line=dict(width=1, color='rgb(255, 51, 51)'),
            fill='toself',
            legendgroup="LXF A677 Gefitinib",
            name="10th to 90th percentile",
            hoverinfo='name',
            showlegend=False,
            visible = dose_id == 0),  # Only show first dosing regimen
        row=2,
        col=1)

    # Plot 25th percentile to 75th precentile
    fig.add_trace(
        go.Scatter(
            x=np.hstack([times,times[::-1]]),
            y=np.hstack([
                population_results[2, dose_id, 1, :], 
                population_results[2, dose_id, 3, ::-1]]),
            line=dict(width=1, color='rgb(204, 0, 0)'),
            fill='toself',
            legendgroup="LXF A677 Gefitinib",
            name="25th to 75th percentile",
            hoverinfo='name',
            showlegend=False,
            visible = dose_id == 0),  # Only show first dosing regimen
        row=2,
        col=1)

    # Plot Median
    fig.add_trace(
        go.Scatter(
            x=times,
            y=population_results[2, dose_id, 2, :],
            legendgroup="LXF A677 Gefitinib",
            name="LXF A677 Gefitinib",
            hovertemplate=
                    "<b></b>Median<br>" +
                    "Population size: %d<br>" % N +
                    "Time: %{x:.02f} day<br>" +
                    "Tumour volume: %{y:.02f} cm^3<br>" +
                    "<extra></extra>",
            line=dict(color='rgb(204, 0, 0)'),
            visible = dose_id == 0),  # Only show first dosing regimen
        row=2,
        col=1)

# Set figure size
fig.update_layout(
    autosize=True,
    template="plotly_white")

# Set X axis label
fig.update_xaxes(title_text=r'$\text{Time in day}$', row=2, col=1)

# Set Y axes labels
fig.update_yaxes(title_text=r'$\text{Tumour volume in cm}^3$', row=2, col=1)
fig.update_yaxes(title_text=r'$\text{Amount in mg}$', row=1, col=1)

# Add switch between linear and log y-scale, and switches between dosing regimens
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "right",
            buttons=list([
                dict(
                    args=[{"yaxis2.type": "linear"}],
                    label="Linear y-scale",
                    method="relayout"
                ),
                dict(
                    args=[{"yaxis2.type": "log"}],
                    label="Log y-scale",
                    method="relayout"
                )
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=0.0,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
        dict(
            type = "buttons",
            direction = "down",
            buttons=list([
                dict(
                    args=[{"visible": [True]*7 + [False]*7 + [False]*7 + [False]*7}],
                    label="No treatment",
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*7 + [True]*7 + [False]*7 + [False]*7}],
                    label="25 mg/kg/day",
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*7 + [False]*7 + [True]*7 + [False]*7}],
                    label="50 mg/kg/day",
                    method="restyle"
                ),
                dict(
                    args=[{"visible": [False]*7 + [False]*7 + [False]*7 + [True]*7}],
                    label="100 mg/kg/day",
                    method="restyle"
                )
            ]),
            pad={"r": 0, "t": -10},
            showactive=True,
            x=1.07,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

# Position legend
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="left",
    x=1.05))

# Show figure
fig.show()


**Figure 1:** Predicted tumour growth in a population of mice with LXF A677 (lung cancer) implants under different dosing strategies with either Erlotinib or Gefinitib. The presented median tumour volumes in the population (solid lines), as well as other percentiles (shaded areas) where computed for each simulated time point for a population of 1000 'virtual' mice, using the reported estimates and structural-population-error model reported in [1]. The uncertainty in the parameter estimates was not incorporated in the model predictions. As a result, the variation of the predicted tumour growth curves is underestimated. Please use the interactive 'zoom' and 'toggle on/off' features to explore the variation in tumour growth behaviour in the populations.

## Bibliography

- <a name="ref1"> [1] </a> Eigenmann et. al., Combining Nonclinical Experiments with Translational PKPD Modeling to Differentiate Erlotinib and Gefitinib, Mol Cancer Ther (2016)